# Import libraries

In [1]:
# !pip install category_encoders
# !pip install lightgbm
# !pip install pillow
# !pip install opencv-python
# !pip install pydicom
# !pip install lightgbm
# !pip install xgboost
# !pip install hyperopt
from lightgbm import LGBMRegressor
import os
from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
from functools import partial
from sklearn import preprocessing
import numpy as np
import pandas as pd
import random
import math
from sklearn.ensemble import GradientBoostingRegressor

from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold
from sklearn.metrics import mean_squared_error
# import category_encoders as ce

from PIL import Image
import cv2
import pydicom

import torch

import lightgbm as lgb
from sklearn.linear_model import Ridge

import warnings
warnings.filterwarnings("ignore")

# !pip install hyperopt
import numpy as np
import pandas as pd
from functools import partial
import xgboost as xgb
from numpy.ma import MaskedArray
import sklearn.utils.fixes
sklearn.utils.fixes.MaskedArray = MaskedArray

import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow.keras.layers as Layers
import tensorflow.keras.models as Models
import warnings
warnings.filterwarnings('ignore') #Ignore "future" warnings and Data-Frame-Slicing warnings.

from sklearn import metrics
from sklearn import ensemble
from sklearn import model_selection
import hyperopt
from hyperopt import hp, fmin, tpe, Trials
from hyperopt.pyll.base import scope

def seed_everything(seed = 2020): 
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    
seed_everything(42)

# Config

In [2]:
ROOT = '../input/osic-pulmonary-fibrosis-progression'

# Read data

## Train data cleanup

In [3]:
train_df = pd.read_csv(f'{ROOT}/train.csv')
print(f'Train data has {train_df.shape[0]} rows and {train_df.shape[1]} columnns and looks like this:')
train_df.sample(10)

Train data has 1549 rows and 7 columnns and looks like this:


,Patient,Weeks,FVC,Percent,Age,Sex,SmokingStatus
350,ID00099637202206203080121,102,2833,79.382425,68,Male,Ex-smoker
650,ID00169637202238024117706,40,2486,105.009715,66,Female,Never smoked
1178,ID00329637202285906759848,43,2848,67.264998,69,Male,Ex-smoker
271,ID00077637202199102000916,6,3121,80.804681,70,Male,Ex-smoker
162,ID00047637202184938901501,6,3370,91.476656,68,Male,Ex-smoker
968,ID00251637202267455595113,37,1937,58.049628,88,Male,Ex-smoker
59,ID00015637202177877247924,40,2697,69.725957,71,Male,Ex-smoker
306,ID00089637202204675567570,7,2478,57.967624,63,Male,Never smoked
1511,ID00419637202311204720264,43,2793,64.911221,73,Male,Ex-smoker
462,ID00124637202217596410344,72,3454,86.194849,60,Male,Ex-smoker


In [4]:
dupRows_df = train_df[train_df.duplicated(subset = ['Patient', 'Weeks'], keep = False )]
dupRows_df

,Patient,Weeks,FVC,Percent,Age,Sex,SmokingStatus
166,ID00048637202185016727717,9,1375,60.059404,70,Female,Never smoked
167,ID00048637202185016727717,9,1477,64.514720,70,Female,Never smoked
226,ID00068637202190879923934,11,2794,82.633385,73,Male,Ex-smoker
227,ID00068637202190879923934,11,2827,83.609369,73,Male,Ex-smoker
306,ID00089637202204675567570,7,2478,57.967624,63,Male,Never smoked
307,ID00089637202204675567570,7,2539,59.394592,63,Male,Never smoked
465,ID00125637202218590429387,8,2349,53.757781,65,Male,Never smoked
466,ID00125637202218590429387,8,2584,59.135848,65,Male,Never smoked
750,ID00199637202248141386743,5,2672,64.728682,62,Male,Ex-smoker
751,ID00199637202248141386743,5,2683,64.995155,62,Male,Ex-smoker


In [5]:
train_df_tmp1 = train_df.groupby(['Patient', 'Weeks']).mean().reset_index()
train_df_tmp2 = train_df[['Patient', 'Weeks', 'Sex', 'SmokingStatus']].drop_duplicates().reset_index(drop=True)
print(train_df_tmp1.shape)
print(train_df_tmp2.shape)
df_train = pd.merge(train_df_tmp1, train_df_tmp2, on=["Patient", "Weeks"], how="inner")
print(df_train.shape)
df_train.head()

(1542, 5)
(1542, 4)
(1542, 7)


,Patient,Weeks,FVC,Percent,Age,Sex,SmokingStatus
0,ID00007637202177411956430,-4,2315.0,58.253649,79.0,Male,Ex-smoker
1,ID00007637202177411956430,5,2214.0,55.712129,79.0,Male,Ex-smoker
2,ID00007637202177411956430,7,2061.0,51.862104,79.0,Male,Ex-smoker
3,ID00007637202177411956430,9,2144.0,53.950679,79.0,Male,Ex-smoker
4,ID00007637202177411956430,11,2069.0,52.063412,79.0,Male,Ex-smoker


## Test data

In [6]:
test_df = pd.read_csv(f'{ROOT}/test.csv')
print(f'Test data has {test_df.shape[0]} rows and {test_df.shape[1]} columnns, has no duplicates and looks like this:')
test_df.head()

Test data has 5 rows and 7 columnns, has no duplicates and looks like this:


,Patient,Weeks,FVC,Percent,Age,Sex,SmokingStatus
0,ID00419637202311204720264,6,3020,70.186855,73,Male,Ex-smoker
1,ID00421637202311550012437,15,2739,82.045291,68,Male,Ex-smoker
2,ID00422637202311677017371,6,1930,76.672493,73,Male,Ex-smoker
3,ID00423637202312137826377,17,3294,79.258903,72,Male,Ex-smoker
4,ID00426637202313170790466,0,2925,71.824968,73,Male,Never smoked


## Submission data

In [7]:
sub_df = pd.read_csv(f"{ROOT}/sample_submission.csv")

print(f"The sample submission contains: {sub_df.shape[0]} rows and {sub_df.shape[1]} columns.")


The sample submission contains: 730 rows and 3 columns.


In [8]:
# split Patient_Week Column and re-arrage columns
sub_df[['Patient','Weeks']] = sub_df.Patient_Week.str.split("_",expand = True)
sub_df =  sub_df[['Patient','Weeks','Confidence', 'Patient_Week']]
sub_df = sub_df.merge(test_df.drop('Weeks', axis = 1), on = "Patient")
sub_df.head(20)

,Patient,Weeks,Confidence,Patient_Week,FVC,Percent,Age,Sex,SmokingStatus
0,ID00419637202311204720264,-12,100,ID00419637202311204720264_-12,3020,70.186855,73,Male,Ex-smoker
1,ID00419637202311204720264,-11,100,ID00419637202311204720264_-11,3020,70.186855,73,Male,Ex-smoker
2,ID00419637202311204720264,-10,100,ID00419637202311204720264_-10,3020,70.186855,73,Male,Ex-smoker
3,ID00419637202311204720264,-9,100,ID00419637202311204720264_-9,3020,70.186855,73,Male,Ex-smoker
4,ID00419637202311204720264,-8,100,ID00419637202311204720264_-8,3020,70.186855,73,Male,Ex-smoker
5,ID00419637202311204720264,-7,100,ID00419637202311204720264_-7,3020,70.186855,73,Male,Ex-smoker
6,ID00419637202311204720264,-6,100,ID00419637202311204720264_-6,3020,70.186855,73,Male,Ex-smoker
7,ID00419637202311204720264,-5,100,ID00419637202311204720264_-5,3020,70.186855,73,Male,Ex-smoker
8,ID00419637202311204720264,-4,100,ID00419637202311204720264_-4,3020,70.186855,73,Male,Ex-smoker
9,ID00419637202311204720264,-3,100,ID00419637202311204720264_-3,3020,70.186855,73,Male,Ex-smoker


## ADS prep

In [9]:
# introduce a column to indicate the source (train/test) for the data
train_df['Source'] = 'train'
sub_df['Source'] = 'test'

data_df = train_df.append([sub_df])
# data_df.reset_index(inplace = True)
print(data_df.shape)
data_df.head()

(2279, 10)


,Patient,Weeks,FVC,Percent,Age,Sex,SmokingStatus,Source,Confidence,Patient_Week
0,ID00007637202177411956430,-4,2315,58.253649,79,Male,Ex-smoker,train,NaN,NaN
1,ID00007637202177411956430,5,2214,55.712129,79,Male,Ex-smoker,train,NaN,NaN
2,ID00007637202177411956430,7,2061,51.862104,79,Male,Ex-smoker,train,NaN,NaN
3,ID00007637202177411956430,9,2144,53.950679,79,Male,Ex-smoker,train,NaN,NaN
4,ID00007637202177411956430,11,2069,52.063412,79,Male,Ex-smoker,train,NaN,NaN


In [10]:
sub_df

,Patient,Weeks,Confidence,Patient_Week,FVC,Percent,Age,Sex,SmokingStatus,Source
0,ID00419637202311204720264,-12,100,ID00419637202311204720264_-12,3020,70.186855,73,Male,Ex-smoker,test
1,ID00419637202311204720264,-11,100,ID00419637202311204720264_-11,3020,70.186855,73,Male,Ex-smoker,test
2,ID00419637202311204720264,-10,100,ID00419637202311204720264_-10,3020,70.186855,73,Male,Ex-smoker,test
3,ID00419637202311204720264,-9,100,ID00419637202311204720264_-9,3020,70.186855,73,Male,Ex-smoker,test
4,ID00419637202311204720264,-8,100,ID00419637202311204720264_-8,3020,70.186855,73,Male,Ex-smoker,test
...,...,...,...,...,...,...,...,...,...,...
725,ID00426637202313170790466,129,100,ID00426637202313170790466_129,2925,71.824968,73,Male,Never smoked,test
726,ID00426637202313170790466,130,100,ID00426637202313170790466_130,2925,71.824968,73,Male,Never smoked,test
727,ID00426637202313170790466,131,100,ID00426637202313170790466_131,2925,71.824968,73,Male,Never smoked,test
728,ID00426637202313170790466,132,100,ID00426637202313170790466_132,2925,71.824968,73,Male,Never smoked,test


In [11]:
def get_baseline_week(df):
    # make a copy to not change original df    
    _df = df.copy()
    # ensure all Weeks values are INT and not accidentaly saved as string
    _df['Weeks'] = _df['Weeks'].astype(int)
    # as test data is containing all weeks, 
    _df.loc[_df.Source == 'test','min_week'] = np.nan
    _df["min_week"] = _df.groupby('Patient')['Weeks'].transform('min')
    _df['baselined_week'] = _df['Weeks'] - _df['min_week']
    
    return _df

In [12]:
data_df = get_baseline_week(data_df)
data_df.head()

,Patient,Weeks,FVC,Percent,Age,Sex,SmokingStatus,Source,Confidence,Patient_Week,min_week,baselined_week
0,ID00007637202177411956430,-4,2315,58.253649,79,Male,Ex-smoker,train,NaN,NaN,-4,0
1,ID00007637202177411956430,5,2214,55.712129,79,Male,Ex-smoker,train,NaN,NaN,-4,9
2,ID00007637202177411956430,7,2061,51.862104,79,Male,Ex-smoker,train,NaN,NaN,-4,11
3,ID00007637202177411956430,9,2144,53.950679,79,Male,Ex-smoker,train,NaN,NaN,-4,13
4,ID00007637202177411956430,11,2069,52.063412,79,Male,Ex-smoker,train,NaN,NaN,-4,15


In [13]:
def get_baseline_FVC(df):
    # same as above
    _df = df.copy()
    base = _df.loc[_df.Weeks == _df.min_week]
    base = base[['Patient','FVC']].copy()
    base.columns = ['Patient','base_FVC']
    
    # add a row which contains the cumulated sum of rows for each patient
    base['nb'] = 1
    base['nb'] = base.groupby('Patient')['nb'].transform('cumsum')
    
    # drop all except the first row for each patient (= unique rows!), containing the min_week
    base = base[base.nb == 1]
    base.drop('nb', axis = 1, inplace = True)
    
    # merge the rows containing the base_FVC on the original _df
    _df = _df.merge(base, on = 'Patient', how = 'left')    
    _df.drop(['min_week'], axis = 1)
    
    return _df

In [14]:
data_df = get_baseline_FVC(data_df)
data_df.head()
data_df.columns

Index(['Patient', 'Weeks', 'FVC', 'Percent', 'Age', 'Sex', 'SmokingStatus',
       'Source', 'Confidence', 'Patient_Week', 'min_week', 'baselined_week',
       'base_FVC'],
      dtype='object')

## Train data K fold prepm

In [15]:
TARGET = 'FVC'
N_FOLDS = 4

In [16]:
df_train = data_df[data_df['Source']=="train"].reset_index(drop=True)
sub_df = data_df[data_df['Source']=="test"].reset_index(drop=True)
group_k_fold = GroupKFold(n_splits=N_FOLDS)
groups = df_train['Patient'].values
for fold, (train_index, valid_index) in enumerate(group_k_fold.split(df_train, df_train[TARGET], groups)):
    df_train.loc[valid_index, 'fold'] = int(fold)
df_train['kfold'] = df_train['fold'].astype(int)
df_train.head()
df_train.to_csv(f'{ROOT}/ads_4_folds.csv', index=False)
sub_df.to_csv(f'{ROOT}/sub_df.csv', index=False)

In [17]:
df_train

,Patient,Weeks,FVC,Percent,Age,Sex,SmokingStatus,Source,Confidence,Patient_Week,min_week,baselined_week,base_FVC,fold,kfold
0,ID00007637202177411956430,-4,2315,58.253649,79,Male,Ex-smoker,train,NaN,NaN,-4,0,2315,1.0,1
1,ID00007637202177411956430,5,2214,55.712129,79,Male,Ex-smoker,train,NaN,NaN,-4,9,2315,1.0,1
2,ID00007637202177411956430,7,2061,51.862104,79,Male,Ex-smoker,train,NaN,NaN,-4,11,2315,1.0,1
3,ID00007637202177411956430,9,2144,53.950679,79,Male,Ex-smoker,train,NaN,NaN,-4,13,2315,1.0,1
4,ID00007637202177411956430,11,2069,52.063412,79,Male,Ex-smoker,train,NaN,NaN,-4,15,2315,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1544,ID00426637202313170790466,13,2712,66.594637,73,Male,Never smoked,train,NaN,NaN,-12,25,2925,1.0,1
1545,ID00426637202313170790466,19,2978,73.126412,73,Male,Never smoked,train,NaN,NaN,-12,31,2925,1.0,1
1546,ID00426637202313170790466,31,2908,71.407524,73,Male,Never smoked,train,NaN,NaN,-12,43,2925,1.0,1
1547,ID00426637202313170790466,43,2975,73.052745,73,Male,Never smoked,train,NaN,NaN,-12,55,2925,1.0,1


# Modeling

In [18]:
df = pd.read_csv(f'{ROOT}/ads_4_folds.csv')
sub_df = pd.read_csv(f'{ROOT}/sub_df.csv')
print(df.shape)
df.head()

(1549, 15)


,Patient,Weeks,FVC,Percent,Age,Sex,SmokingStatus,Source,Confidence,Patient_Week,min_week,baselined_week,base_FVC,fold,kfold
0,ID00007637202177411956430,-4,2315,58.253649,79,Male,Ex-smoker,train,NaN,NaN,-4,0,2315,1.0,1
1,ID00007637202177411956430,5,2214,55.712129,79,Male,Ex-smoker,train,NaN,NaN,-4,9,2315,1.0,1
2,ID00007637202177411956430,7,2061,51.862104,79,Male,Ex-smoker,train,NaN,NaN,-4,11,2315,1.0,1
3,ID00007637202177411956430,9,2144,53.950679,79,Male,Ex-smoker,train,NaN,NaN,-4,13,2315,1.0,1
4,ID00007637202177411956430,11,2069,52.063412,79,Male,Ex-smoker,train,NaN,NaN,-4,15,2315,1.0,1


## filter relevant columns

In [19]:
df = df[['Patient', 'FVC', 'Age', 'Sex', 'SmokingStatus', 'baselined_week', 'base_FVC', 'kfold']]
df.head()

,Patient,FVC,Age,Sex,SmokingStatus,baselined_week,base_FVC,kfold
0,ID00007637202177411956430,2315,79,Male,Ex-smoker,0,2315,1
1,ID00007637202177411956430,2214,79,Male,Ex-smoker,9,2315,1
2,ID00007637202177411956430,2061,79,Male,Ex-smoker,11,2315,1
3,ID00007637202177411956430,2144,79,Male,Ex-smoker,13,2315,1
4,ID00007637202177411956430,2069,79,Male,Ex-smoker,15,2315,1


In [20]:
print(df['kfold'].unique())
df['kfold'].value_counts()

[1 0 3 2]


0    388
3    387
2    387
1    387
Name: kfold, dtype: int64

In [21]:
features = [f for f in df.columns if f not in ["Patient", "kfold", "FVC"]]
print(features)
cat_features = ["Sex", "SmokingStatus"]

for col in cat_features:
    lbl = preprocessing.LabelEncoder()
    lbl.fit(df[col])
    df.loc[:, col] = lbl.transform(df[col])
    sub_df.loc[:, col] = lbl.transform(sub_df[col])

['Age', 'Sex', 'SmokingStatus', 'baselined_week', 'base_FVC']


## Hyperopt to identify best parameters

In [22]:
def compute_quantile_loss(y_true, y_pred, quantile):
    """
    
    Parameters
    ----------
    y_true : 1d ndarray
        Target value.
        
    y_pred : 1d ndarray
        Predicted value.
        
    quantile : float, 0. ~ 1.
        Quantile to be evaluated, e.g., 0.5 for median.
    """
    residual = y_true - y_pred
    return np.maximum(quantile * residual, (quantile - 1) * residual)

def optimize(params, x, y, groups, quantile):

    # initialize model with current parameters
    model = GradientBoostingRegressor(**params)

    group_k_fold = GroupKFold(n_splits=N_FOLDS)

    # initialize accuracy list
#     rmse_errors = []
    fold_quantile_losses = []
    
    for idx in group_k_fold.split(x, y, groups):
        train_idx, test_idx = idx[0], idx[1]
        xtrain = x[train_idx]
        ytrain = y[train_idx]

        xtest = x[test_idx]
        ytest = y[test_idx]

        # fit model on train data
        model.fit(xtrain, ytrain)

        # create predictions
        preds = model.predict(xtest)
        fold_quantile_loss = compute_quantile_loss(ytest, preds, quantile)
#         fold_rmse = np.sqrt(mean_squared_error(ytest, preds))
        fold_quantile_losses.append(np.mean(fold_quantile_loss))
    
    return np.mean(fold_quantile_losses)

In [23]:
quantiles = [0.15, 0.5, 0.85]

best_params_list = []
for quantile in quantiles:
    
    
#     param_space = {
#                'objective': 'quantile',
#                'max_depth':  hp.choice('max_depth', [4, 5, 6, 7, 8, 10, 12, 15]),
#                'num_leaves': hp.choice('num_leaves', [100, 250, 500, 650, 750, 1000,1300]),
#                'learning_rate': hp.loguniform('learning_rate',-3,1),
#                'alpha': quantile}
    
    param_space = {
            'n_estimators': hp.randint('n_estimators',1000) ,
            'learning_rate': hp.loguniform('learning_rate',-3,1),
            'loss': 'quantile',
            'alpha': quantile

    }

    groups = df.Patient.values
    X = df[features].values
    y = df[TARGET].values
    optimization_function = partial(optimize, x=X, y=y, groups = groups, quantile=quantile)
    # initialize Trials to keep logging information
    trials = Trials()

    # run hyperopt
    print(f"Starting HP tuning using hyperopt for quantile --> {quantile}")
    hopt = fmin(fn=optimization_function,
    space=param_space,
    algo=tpe.suggest,
    max_evals=25,
    trials=trials)
    
    print(f"Quantile --> {quantile}")
    print(hopt)
    best_params = dict(zip(list(hopt.keys()), list(hopt.values())))
    print(best_params)
    best_params_list.append(best_params)

Starting HP tuning using hyperopt for quantile --> 0.15
100%|██████████| 25/25 [01:04<00:00,  2.59s/trial, best loss: 60.06060924181171] 
Quantile --> 0.15
{'learning_rate': 0.11101572493513455, 'n_estimators': 476}
{'learning_rate': 0.11101572493513455, 'n_estimators': 476}
Starting HP tuning using hyperopt for quantile --> 0.5
100%|██████████| 25/25 [00:59<00:00,  2.40s/trial, best loss: 86.44181018961174]
Quantile --> 0.5
{'learning_rate': 0.06423182032805837, 'n_estimators': 391}
{'learning_rate': 0.06423182032805837, 'n_estimators': 391}
Starting HP tuning using hyperopt for quantile --> 0.85
100%|██████████| 25/25 [01:13<00:00,  2.94s/trial, best loss: 52.64149547714133]
Quantile --> 0.85
{'learning_rate': 0.052665155484878726, 'n_estimators': 405}
{'learning_rate': 0.052665155484878726, 'n_estimators': 405}


In [30]:
def run(fold):
    df_train = df[df.kfold !=fold].reset_index(drop=True)
    # get validation data
    df_valid = df[df.kfold ==fold].reset_index(drop=True)
    print(df_valid.shape)
    # get training data
    x_train = df_train[features]
    # get validation data
    x_valid = df_valid[features]
    x_test = sub_df[features]
    for q_ in range(len(best_params_list)):
        best_model = GradientBoostingRegressor(loss='quantile',
                          alpha=quantiles[q_],
                          n_estimators=best_params_list[q_]['n_estimators'],
                          learning_rate=best_params_list[q_]['learning_rate'])
        
#         best_model = LGBMRegressor(objective='quantile',
#                                    alpha=quantiles[q_],
#                                    learning_rate=best_params_list[q_]['learning_rate'],
#                                    max_depth=best_params_list[q_]['max_depth'],
#                                    num_leaves=best_params_list[q_]['num_leaves'],
                                   
#                                   )
        best_model.fit(x_train, df_train.FVC.values)
        pred = best_model.predict(x_valid).round(2)
        df_valid.loc[:, str('pred_fold_' + str(fold) + "_" + str(quantiles[q_]))] = pred
        pred_test = best_model.predict(x_test).round(2)
        sub_df.loc[:, str('pred_fold_' + str(fold) + "_" + str(quantiles[q_]))] = pred_test
    return df_valid, sub_df


In [31]:
df_tmp_0, sub_df_0 = run(0)
df_tmp_1, sub_df_1 = run(1)
df_tmp_2, sub_df_2 = run(2)
df_tmp_3, sub_df_3 = run(3)


(388, 8)
(387, 8)
(387, 8)
(387, 8)


In [32]:
print(df_tmp_1.columns)
print(sub_df_1.columns)


Index(['Patient', 'FVC', 'Age', 'Sex', 'SmokingStatus', 'baselined_week',
       'base_FVC', 'kfold', 'pred_fold_1_0.15', 'pred_fold_1_0.5',
       'pred_fold_1_0.85'],
      dtype='object')
Index(['Patient', 'Weeks', 'FVC', 'Percent', 'Age', 'Sex', 'SmokingStatus',
       'Source', 'Confidence', 'Patient_Week', 'min_week', 'baselined_week',
       'base_FVC', 'pred_fold_0_0.15', 'pred_fold_0_0.5', 'pred_fold_0_0.85',
       'pred_fold_1_0.15', 'pred_fold_1_0.5', 'pred_fold_1_0.85',
       'pred_fold_2_0.15', 'pred_fold_2_0.5', 'pred_fold_2_0.85',
       'pred_fold_3_0.15', 'pred_fold_3_0.5', 'pred_fold_3_0.85'],
      dtype='object')


In [33]:
def score(df, col_50, col_15, col_85, true):
    
    df['Confidence'] = df[col_85] - df[col_15]
    df['sigma_clipped'] = df['Confidence'].apply(lambda x: max(x, 70))
    df['diff'] = abs(df[true] - df[col_50])
    df['delta'] = df['diff'].apply(lambda x: min(x, 1000))
    df['score'] = -math.sqrt(2)*df['delta']/df['sigma_clipped'] - np.log(math.sqrt(2)*df['sigma_clipped'])
    return df['score']

 

In [34]:
df_tmp_0['fold0_score'] = score(df_tmp_0, 'pred_fold_0_0.5', 'pred_fold_0_0.15', 'pred_fold_0_0.85', 'FVC' )
df_tmp_1['fold1_score'] = score(df_tmp_1, 'pred_fold_1_0.5', 'pred_fold_1_0.15', 'pred_fold_1_0.85', 'FVC' )
df_tmp_2['fold2_score'] = score(df_tmp_2, 'pred_fold_2_0.5', 'pred_fold_2_0.15', 'pred_fold_2_0.85', 'FVC' )
df_tmp_3['fold3_score'] = score(df_tmp_3, 'pred_fold_3_0.5', 'pred_fold_3_0.15', 'pred_fold_3_0.85', 'FVC' )

In [35]:
print(df_tmp_0.sort_values(['Patient', 'baselined_week']).groupby('Patient').tail(3).reset_index(drop=True)['score'].mean())
print(df_tmp_1.sort_values(['Patient', 'baselined_week']).groupby('Patient').tail(3).reset_index(drop=True)['score'].mean())
print(df_tmp_2.sort_values(['Patient', 'baselined_week']).groupby('Patient').tail(3).reset_index(drop=True)['score'].mean())
print(df_tmp_3.sort_values(['Patient', 'baselined_week']).groupby('Patient').tail(3).reset_index(drop=True)['score'].mean())


-7.25633840062985
-7.0308747260893005
-7.285724503335688
-7.171356263273551


In [36]:
sub_df_0['Confidence'] = ((sub_df_0['pred_fold_3_0.85'] + sub_df_0['pred_fold_2_0.85'] + \
                          sub_df_0['pred_fold_1_0.85'] + sub_df_0['pred_fold_0_0.85']) - \
                         (sub_df_0['pred_fold_3_0.15'] + sub_df_0['pred_fold_2_0.15'] + \
                         sub_df_0['pred_fold_1_0.15'] + sub_df_0['pred_fold_0_0.15']))/(4)

sub_df_0["FVC"] = ((sub_df_0['pred_fold_0_0.5'] + sub_df_0['pred_fold_1_0.5'] + \
                  sub_df_0['pred_fold_2_0.5'] + sub_df_0['pred_fold_3_0.5']))/4

In [37]:
final_submission = sub_df_0[["Patient_Week","FVC","Confidence"]]
print(final_submission.shape)
# final_submission.to_csv("submission.csv", index = False)
final_submission.head()

(730, 3)


,Patient_Week,FVC,Confidence
0,ID00419637202311204720264_-12,2934.475,111.8950
1,ID00419637202311204720264_-11,2883.800,210.2625
2,ID00419637202311204720264_-10,2879.780,222.0125
3,ID00419637202311204720264_-9,2876.655,240.9875
4,ID00419637202311204720264_-8,2871.700,247.0650


In [ ]:
-7.225472825440669
-6.977682840522279
-7.38676677148804
-7.100856808916671